In [1]:
import numpy as np
import networkx as nx
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [2]:
n=5 #Prime router number
N=10 #regular router number
m=3 #shortest path number
V=10000

In [3]:
file_best_clustering='full_graph_best_clustering_{}_{}.txt'.format(n,N)
file_name_cluster_graph='cluster_graph_{}_{}.txt'.format(n,N)
file_name_shortest_path='full_graph_shortest_path_{}_{}.txt'.format(n,N)
file_name_prime_routers_shortest_path='prime_graph_shortest_path_{}_{}.txt'.format(n,N)
file_name_prime_graph='prime_graph_{}_{}.txt'.format(n,N)

# Reading graph

In [4]:
con=[[] for _ in range(n+N+1)]
edges=set()

In [5]:
with open('test/'+file_name_cluster_graph, 'r') as f:
    
    n=int(f.readline())
    next(f)
    
    #f.write('\n')
    for line in f:
        #print('line',line)
        x,y,w=map(float,line.split())
        x,y=int(x),int(y)
        #print(x,y,w)
        con[x].append((y,w))
        #con[y].append((x,w))
        edges.add((x,y,w))

           
    f.close()

In [6]:
#con
edges

{(1, 3, 0.23142470169810703),
 (1, 5, 0.7536451376844908),
 (1, 6, 0.4638092727662523),
 (1, 7, 0.9581887310477116),
 (1, 8, 0.11962100470551662),
 (1, 13, 0.10916629732677441),
 (1, 14, 0.9991049222877524),
 (2, 3, 0.032934312265980425),
 (2, 5, 0.6369470115993228),
 (2, 6, 0.9557802366404332),
 (2, 9, 0.3430049053278973),
 (2, 14, 0.7731763192800118),
 (3, 1, 0.23142470169810703),
 (3, 2, 0.032934312265980425),
 (3, 4, 0.501869175806194),
 (3, 5, 0.882580654303218),
 (3, 6, 0.013962574310672937),
 (3, 7, 0.3114343887508788),
 (3, 11, 0.6107640985517377),
 (3, 12, 0.7868252034308975),
 (3, 14, 0.9658427179915304),
 (4, 3, 0.501869175806194),
 (4, 6, 0.7856803587257708),
 (4, 8, 0.6293629338431845),
 (4, 9, 0.15686755357253557),
 (4, 13, 0.959083650317909),
 (5, 1, 0.7536451376844908),
 (5, 2, 0.6369470115993228),
 (5, 3, 0.882580654303218),
 (5, 11, 0.527469403352322),
 (5, 12, 0.9106297271042973),
 (5, 13, 0.2205953804350782),
 (6, 1, 0.4638092727662523),
 (6, 2, 0.9557802366404332),

In [7]:
con[1]

[(3, 0.23142470169810703),
 (3, 0.23142470169810703),
 (5, 0.7536451376844908),
 (5, 0.7536451376844908),
 (6, 0.4638092727662523),
 (6, 0.4638092727662523),
 (7, 0.9581887310477116),
 (7, 0.9581887310477116),
 (8, 0.11962100470551662),
 (8, 0.11962100470551662),
 (13, 0.10916629732677441),
 (13, 0.10916629732677441),
 (14, 0.9991049222877524),
 (14, 0.9991049222877524)]

# Reading shortest paths

In [8]:
sh_path=[[] for _ in range(n+N+1)]

for i in range(n+N+1):
    sh_path[i]=[[] for _ in range(n+N+1)]

In [9]:
with open('test/'+file_name_shortest_path, 'r') as f:
    
    n,N=map(int,f.readline().split())
    
    for i in range(1,n+N+1):
        for j in range(1,n+N+1):
            if i!=j:
                x,y=map(int,f.readline().split())
                #print('x,y',x,y)
                
                while True:
                    line1=f.readline()
                    #print('line1',line1)
                    
                    if not line1.strip():
                        break

                    line2=f.readline()
                    
                    w=float(line1)
                    path=list(map(int,line2.split()))
                    #print('path',path)
                    #print('w',w)

                    sh_path[i][j].append((w,path))


In [10]:
sh_path[1]

[[],
 [],
 [(0.26435901396408745, [1, 3, 2])],
 [(0.23142470169810703, [1, 3])],
 [(0.5321960537989762, [1, 13, 9, 4])],
 [(0.3297616777618526, [1, 13, 5])],
 [(0.24538727600877996, [1, 3, 6])],
 [(0.1394212321147007, [1, 13, 7])],
 [(0.11710558816195982, [1, 13, 8])],
 [(0.3753285002264406, [1, 13, 9])],
 [(0.6085044255795418, [1, 3, 6, 10])],
 [(0.8421888002498448, [1, 3, 11])],
 [(0.5343118449864267, [1, 13, 7, 12])],
 [(0.10916629732677441, [1, 13])],
 [(0.33897221882009543, [1, 3, 6, 14])],
 [(0.7895201727717907, [1, 13, 8, 15])],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

# Figuring out the coefficients gamma

In [11]:
edges_to_path=set()

In [12]:
def belong_to_path():
   
    for i in range(1,N+1):
        for j in range(N+1,n+N+1):

            #print('i,j,',i,j)
            path=sh_path[i][j][0][1]
            l=len(path)

            for k in range(l-1):

                #if path[k]>n and path[k+1]>n:
                #    continue

                edges_to_path.add((path[k],path[k+1],i,j))

In [13]:
belong_to_path()

In [14]:
sh_path[1][2][0][1]

[1, 3, 2]

In [15]:
edges_to_path

{(1, 3, 1, 11),
 (1, 3, 1, 14),
 (1, 13, 1, 12),
 (1, 13, 1, 13),
 (1, 13, 1, 15),
 (1, 13, 2, 13),
 (1, 13, 3, 13),
 (2, 3, 2, 11),
 (2, 3, 2, 12),
 (2, 3, 2, 13),
 (2, 3, 2, 14),
 (2, 3, 2, 15),
 (3, 1, 2, 13),
 (3, 1, 3, 13),
 (3, 6, 1, 14),
 (3, 6, 2, 14),
 (3, 6, 2, 15),
 (3, 6, 3, 14),
 (3, 6, 3, 15),
 (3, 7, 2, 12),
 (3, 7, 3, 12),
 (3, 11, 1, 11),
 (3, 11, 2, 11),
 (3, 11, 3, 11),
 (3, 11, 6, 11),
 (3, 11, 10, 11),
 (4, 9, 4, 11),
 (4, 9, 4, 12),
 (4, 9, 4, 13),
 (4, 9, 4, 14),
 (4, 9, 4, 15),
 (5, 11, 5, 11),
 (5, 11, 7, 11),
 (5, 11, 8, 11),
 (5, 13, 5, 12),
 (5, 13, 5, 13),
 (5, 13, 5, 14),
 (5, 13, 5, 15),
 (6, 3, 6, 11),
 (6, 3, 10, 11),
 (6, 7, 6, 12),
 (6, 7, 6, 13),
 (6, 14, 1, 14),
 (6, 14, 2, 14),
 (6, 14, 3, 14),
 (6, 14, 5, 14),
 (6, 14, 6, 14),
 (6, 14, 7, 14),
 (6, 14, 8, 14),
 (6, 14, 10, 14),
 (6, 15, 2, 15),
 (6, 15, 3, 15),
 (6, 15, 6, 15),
 (7, 6, 5, 14),
 (7, 6, 7, 14),
 (7, 6, 8, 14),
 (7, 12, 1, 12),
 (7, 12, 2, 12),
 (7, 12, 3, 12),
 (7, 12, 4, 12),
 (7, 

# Collecting terms

In [16]:
def mult(a):
    d={}
    
    for q in a:
        for e in q['term']:
            for e2 in q['term']:
                
                x,y=e
                z,t=e2
                
                if x<z:
                    x,z=z,x
                    y,t=t,y
                
                coeff=q['coeff']
                            
                if x!='C' and z!='C':
                    d[(x,z)]=d[(x,z)]+coeff*y*t if (x,z) in d else coeff*y*t
                    
                if x=='C' and z!='C':
                    d[(z,z)]=d[(z,z)]+coeff*y*t if (z,z) in d else coeff*y*t
                    
                if x!='C' and z=='C':
                    d[(x,x)]=d[(x,x)]+coeff*y*t if (x,x) in d else coeff*y*t
                    
        #print(d)
        

                    
    return d

# Example

In [17]:
form=[{'coeff':1, 'term':[('y2',1)]}, {'coeff':4, 'term':[('x1',1), ('x2',1), ('y2',1)]}, {'coeff':1, 'term':[('x2',1)]}, {'coeff':1, 'term':[('y1',1)]}, 
      {'coeff':1, 'term':[('x1',1), ('y2',1)]}, {'coeff':9, 'term':[('x1',1), ('y1',1)]}, {'coeff':25, 'term':[('x1',1), ('x2',1), ('C',-1)]}, {'coeff':25, 'term':[('y1',1), ('y2',1), ('C',-1)]}]

In [18]:
Q=mult(form) #we ignore a constant term

In [19]:
Q #we are ignoring the constant term and using the fact that x^2=x, x in {0,1}

{('y2', 'y2'): -19,
 ('x1', 'x1'): -11,
 ('x2', 'x1'): 58,
 ('y2', 'x1'): 10,
 ('x2', 'x2'): -20,
 ('y2', 'x2'): 8,
 ('y1', 'y1'): -15,
 ('y1', 'x1'): 18,
 ('y2', 'y1'): 50}

# Real network example

In [20]:
form=[0]*(n+1)

for i in range(n+1):
    form[i]=[0]*(n+1)

In [21]:
B=np.random.rand(N+1,1)

In [22]:
form=[]
ys=set()

In [23]:
for e in edges:
    x,y,w=e
    
    #if x>N and y>N:
    #    continue
    
    
    term=[]
    coeff=0
    
    for i in range(1,N+1):
        for j in range(N+1,N+n+1):
            if i!=j:
                                 
                if (x,y,i,j) in edges_to_path:
                    coeff=1.0/w
                    name='y'+'_'+str(i)+'_'+str(j)
                    term.append((name,B[i].item()))
                    ys.add(name)
            
            
    if coeff!=0:
        print('x,y',x,y,term)
        form.append({'coeff':coeff, 'term':term})                

x,y 6 7 [('y_6_12', 0.09437519894232782), ('y_6_13', 0.09437519894232782)]
x,y 9 11 [('y_4_11', 0.9567855967924153), ('y_9_11', 0.911453722045956)]
x,y 3 7 [('y_2_12', 0.43780743779171927), ('y_3_12', 0.7546822156485584)]
x,y 1 13 [('y_1_12', 0.203422356510219), ('y_1_13', 0.203422356510219), ('y_1_15', 0.203422356510219), ('y_2_13', 0.43780743779171927), ('y_3_13', 0.7546822156485584)]
x,y 3 1 [('y_2_13', 0.43780743779171927), ('y_3_13', 0.7546822156485584)]
x,y 7 13 [('y_6_13', 0.09437519894232782), ('y_7_11', 0.9575772660094215), ('y_7_13', 0.9575772660094215), ('y_7_15', 0.9575772660094215)]
x,y 5 13 [('y_5_12', 0.4396751426256147), ('y_5_13', 0.4396751426256147), ('y_5_14', 0.4396751426256147), ('y_5_15', 0.4396751426256147)]
x,y 9 14 [('y_4_14', 0.9567855967924153), ('y_9_14', 0.911453722045956)]
x,y 6 3 [('y_6_11', 0.09437519894232782), ('y_10_11', 0.5382350924414845)]
x,y 7 6 [('y_5_14', 0.4396751426256147), ('y_7_14', 0.9575772660094215), ('y_8_14', 0.6639825420405049)]
x,y 10

In [24]:
len(ys)

50

In [25]:
edges

{(1, 3, 0.23142470169810703),
 (1, 5, 0.7536451376844908),
 (1, 6, 0.4638092727662523),
 (1, 7, 0.9581887310477116),
 (1, 8, 0.11962100470551662),
 (1, 13, 0.10916629732677441),
 (1, 14, 0.9991049222877524),
 (2, 3, 0.032934312265980425),
 (2, 5, 0.6369470115993228),
 (2, 6, 0.9557802366404332),
 (2, 9, 0.3430049053278973),
 (2, 14, 0.7731763192800118),
 (3, 1, 0.23142470169810703),
 (3, 2, 0.032934312265980425),
 (3, 4, 0.501869175806194),
 (3, 5, 0.882580654303218),
 (3, 6, 0.013962574310672937),
 (3, 7, 0.3114343887508788),
 (3, 11, 0.6107640985517377),
 (3, 12, 0.7868252034308975),
 (3, 14, 0.9658427179915304),
 (4, 3, 0.501869175806194),
 (4, 6, 0.7856803587257708),
 (4, 8, 0.6293629338431845),
 (4, 9, 0.15686755357253557),
 (4, 13, 0.959083650317909),
 (5, 1, 0.7536451376844908),
 (5, 2, 0.6369470115993228),
 (5, 3, 0.882580654303218),
 (5, 11, 0.527469403352322),
 (5, 12, 0.9106297271042973),
 (5, 13, 0.2205953804350782),
 (6, 1, 0.4638092727662523),
 (6, 2, 0.9557802366404332),

In [26]:
ys

{'y_10_11',
 'y_10_12',
 'y_10_13',
 'y_10_14',
 'y_10_15',
 'y_1_11',
 'y_1_12',
 'y_1_13',
 'y_1_14',
 'y_1_15',
 'y_2_11',
 'y_2_12',
 'y_2_13',
 'y_2_14',
 'y_2_15',
 'y_3_11',
 'y_3_12',
 'y_3_13',
 'y_3_14',
 'y_3_15',
 'y_4_11',
 'y_4_12',
 'y_4_13',
 'y_4_14',
 'y_4_15',
 'y_5_11',
 'y_5_12',
 'y_5_13',
 'y_5_14',
 'y_5_15',
 'y_6_11',
 'y_6_12',
 'y_6_13',
 'y_6_14',
 'y_6_15',
 'y_7_11',
 'y_7_12',
 'y_7_13',
 'y_7_14',
 'y_7_15',
 'y_8_11',
 'y_8_12',
 'y_8_13',
 'y_8_14',
 'y_8_15',
 'y_9_11',
 'y_9_12',
 'y_9_13',
 'y_9_14',
 'y_9_15'}

In [27]:
form

[{'coeff': 3.3212173503327573,
  'term': [('y_6_12', 0.09437519894232782), ('y_6_13', 0.09437519894232782)]},
 {'coeff': 1.0911949744866263,
  'term': [('y_4_11', 0.9567855967924153), ('y_9_11', 0.911453722045956)]},
 {'coeff': 3.210949195465744,
  'term': [('y_2_12', 0.43780743779171927), ('y_3_12', 0.7546822156485584)]},
 {'coeff': 9.160336335367651,
  'term': [('y_1_12', 0.203422356510219),
   ('y_1_13', 0.203422356510219),
   ('y_1_15', 0.203422356510219),
   ('y_2_13', 0.43780743779171927),
   ('y_3_13', 0.7546822156485584)]},
 {'coeff': 4.3210599070124225,
  'term': [('y_2_13', 0.43780743779171927), ('y_3_13', 0.7546822156485584)]},
 {'coeff': 33.052459276794266,
  'term': [('y_6_13', 0.09437519894232782),
   ('y_7_11', 0.9575772660094215),
   ('y_7_13', 0.9575772660094215),
   ('y_7_15', 0.9575772660094215)]},
 {'coeff': 4.533186497503753,
  'term': [('y_5_12', 0.4396751426256147),
   ('y_5_13', 0.4396751426256147),
   ('y_5_14', 0.4396751426256147),
   ('y_5_15', 0.439675142625

In [28]:
for i in range(1,N+1):
    
    term=[]
    
    for j in range(N+1,n+N+1):

        name='y'+'_'+str(i)+'_'+str(j)

        if name in ys:
            term.append((name,1))


    term.append(('C',-1))

    print('term',term)

           
    form.append({'coeff':V, 'term':term})   

                    
                    

term [('y_1_11', 1), ('y_1_12', 1), ('y_1_13', 1), ('y_1_14', 1), ('y_1_15', 1), ('C', -1)]
term [('y_2_11', 1), ('y_2_12', 1), ('y_2_13', 1), ('y_2_14', 1), ('y_2_15', 1), ('C', -1)]
term [('y_3_11', 1), ('y_3_12', 1), ('y_3_13', 1), ('y_3_14', 1), ('y_3_15', 1), ('C', -1)]
term [('y_4_11', 1), ('y_4_12', 1), ('y_4_13', 1), ('y_4_14', 1), ('y_4_15', 1), ('C', -1)]
term [('y_5_11', 1), ('y_5_12', 1), ('y_5_13', 1), ('y_5_14', 1), ('y_5_15', 1), ('C', -1)]
term [('y_6_11', 1), ('y_6_12', 1), ('y_6_13', 1), ('y_6_14', 1), ('y_6_15', 1), ('C', -1)]
term [('y_7_11', 1), ('y_7_12', 1), ('y_7_13', 1), ('y_7_14', 1), ('y_7_15', 1), ('C', -1)]
term [('y_8_11', 1), ('y_8_12', 1), ('y_8_13', 1), ('y_8_14', 1), ('y_8_15', 1), ('C', -1)]
term [('y_9_11', 1), ('y_9_12', 1), ('y_9_13', 1), ('y_9_14', 1), ('y_9_15', 1), ('C', -1)]
term [('y_10_11', 1), ('y_10_12', 1), ('y_10_13', 1), ('y_10_14', 1), ('y_10_15', 1), ('C', -1)]


In [29]:
form

[{'coeff': 3.3212173503327573,
  'term': [('y_6_12', 0.09437519894232782), ('y_6_13', 0.09437519894232782)]},
 {'coeff': 1.0911949744866263,
  'term': [('y_4_11', 0.9567855967924153), ('y_9_11', 0.911453722045956)]},
 {'coeff': 3.210949195465744,
  'term': [('y_2_12', 0.43780743779171927), ('y_3_12', 0.7546822156485584)]},
 {'coeff': 9.160336335367651,
  'term': [('y_1_12', 0.203422356510219),
   ('y_1_13', 0.203422356510219),
   ('y_1_15', 0.203422356510219),
   ('y_2_13', 0.43780743779171927),
   ('y_3_13', 0.7546822156485584)]},
 {'coeff': 4.3210599070124225,
  'term': [('y_2_13', 0.43780743779171927), ('y_3_13', 0.7546822156485584)]},
 {'coeff': 33.052459276794266,
  'term': [('y_6_13', 0.09437519894232782),
   ('y_7_11', 0.9575772660094215),
   ('y_7_13', 0.9575772660094215),
   ('y_7_15', 0.9575772660094215)]},
 {'coeff': 4.533186497503753,
  'term': [('y_5_12', 0.4396751426256147),
   ('y_5_13', 0.4396751426256147),
   ('y_5_14', 0.4396751426256147),
   ('y_5_15', 0.439675142625

In [30]:
Q=mult(form)

In [31]:
Q

{('y_6_12', 'y_6_12'): -9999.94786418748,
 ('y_6_13', 'y_6_12'): 20000.059162028178,
 ('y_6_13', 'y_6_13'): -9999.676031368226,
 ('y_4_11', 'y_4_11'): -9993.165335230155,
 ('y_9_11', 'y_4_11'): 1.9031876223517061,
 ('y_9_11', 'y_9_11'): -9999.09349208017,
 ('y_2_12', 'y_2_12'): -9993.07922290162,
 ('y_3_12', 'y_2_12'): 3.7952330565118775,
 ('y_3_12', 'y_3_12'): -9996.728933036818,
 ('y_1_12', 'y_1_12'): -9998.14841669107,
 ('y_1_13', 'y_1_12'): 20000.758121437502,
 ('y_1_15', 'y_1_12'): 20000.758121437502,
 ('y_2_13', 'y_1_12'): 1.631635823036193,
 ('y_3_13', 'y_1_12'): 2.8125756480325457,
 ('y_1_13', 'y_1_13'): -9999.620939281249,
 ('y_1_15', 'y_1_13'): 20000.758121437502,
 ('y_2_13', 'y_1_13'): 1.631635823036193,
 ('y_3_13', 'y_1_13'): 2.8125756480325457,
 ('y_1_15', 'y_1_15'): -9994.347263956337,
 ('y_2_13', 'y_1_15'): 1.631635823036193,
 ('y_3_13', 'y_1_15'): 2.8125756480325457,
 ('y_2_13', 'y_2_13'): -9991.596019818226,
 ('y_3_13', 'y_2_13'): 8.90865458666239,
 ('y_3_13', 'y_3_13'

In [32]:
len(Q)

270

In [33]:
sampler = EmbeddingComposite(DWaveSampler())

In [34]:
response=sampler.sample_qubo(Q, num_reads=1)

In [35]:
for data in response.data(['sample','energy', 'num_occurrences']):
    
    print(data.sample, 'energy', data.energy, 'num_occurrences', data.num_occurrences) 

{'y_10_11': 0, 'y_10_12': 0, 'y_10_13': 0, 'y_10_14': 1, 'y_10_15': 0, 'y_1_11': 0, 'y_1_12': 0, 'y_1_13': 0, 'y_1_14': 0, 'y_1_15': 1, 'y_2_11': 0, 'y_2_12': 0, 'y_2_13': 0, 'y_2_14': 1, 'y_2_15': 0, 'y_3_11': 0, 'y_3_12': 0, 'y_3_13': 0, 'y_3_14': 1, 'y_3_15': 0, 'y_4_11': 0, 'y_4_12': 1, 'y_4_13': 0, 'y_4_14': 0, 'y_4_15': 0, 'y_5_11': 0, 'y_5_12': 1, 'y_5_13': 0, 'y_5_14': 0, 'y_5_15': 0, 'y_6_11': 0, 'y_6_12': 1, 'y_6_13': 0, 'y_6_14': 0, 'y_6_15': 0, 'y_7_11': 0, 'y_7_12': 0, 'y_7_13': 0, 'y_7_14': 1, 'y_7_15': 0, 'y_8_11': 0, 'y_8_12': 0, 'y_8_13': 1, 'y_8_14': 0, 'y_8_15': 0, 'y_9_11': 1, 'y_9_12': 0, 'y_9_13': 0, 'y_9_14': 0, 'y_9_15': 0} energy -99668.9124379692 num_occurrences 1


# Saving function f

In [36]:
func_f=[0]*(N+1)

In [37]:
for i in range(1,N+1):
    for j in range(N+1,n+N+1):
        name='y_'+str(i)+'_'+str(j)
        #print(data.sample[name])
        
        if data.sample[name]:
            func_f[i]=j
            break
            
    #if not func_f[i]:
    #    func_f[i]=N+1

In [38]:
with open('test/'+file_best_clustering, 'w') as f:
    
    f.write('{} {}\n\n'.format(n,N))

    for i in range(1,N+1):
        f.write('{} {}\n'.format(i,func_f[i]))
        
                    
    f.close()

In [39]:
func_f[1:]

[15, 14, 14, 12, 12, 12, 14, 13, 11, 14]